In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install utils torch numpy einops

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cach

In [3]:
import torch
import os
import numpy as np
from datetime import datetime
import argparse
import sys

%cd /content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC
path = '/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC'
os.chdir(path)
sys.path.append(path)

from utils import _logger, set_requires_grad
from dataloader.dataloader import data_generator
from trainer.trainer import Trainer, model_evaluate
from models.TC import TC
from utils import _calc_metrics, copy_Files
from models.model import base_Model
# Args selections
start_time = datetime.now()

#maybe something to do with (not torch = th) th.randperm

/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC


In [4]:
#drive.flush_and_unmount()

In [5]:
#main
# Args selections
start_time = datetime.now()

parser = argparse.ArgumentParser()

######################## Model parameters ########################
home_dir = os.getcwd()

class Configs:
    class augmentations:
        def __init__(self):
            self.jitter_scale_ratio = 1.1
            self.jitter_ratio = 0.8
            self.max_seg = 8

    class Context_Cont_configs:
        def __init__(self):
            self.temperature = 0.2
            self.use_cosine_similarity = True

    class TC:
        def __init__(self):
            self.hidden_dim = 100
            self.timesteps = 6

    def __init__(self):
        self.input_channels = 1
        self.kernel_size = 8
        self.stride = 1
        self.final_out_channels = 320 # set to 320 according to ts2vec paper about tstcc comparison

        self.num_classes = 5
        self.dropout = 0.35
        self.features_len = 74 #needs changing per dataset based on length of sample

        # training configs
        self.num_epoch = 40 # 200 caused overfitting

        # optimizer parameters
        self.beta1 = 0.9
        self.beta2 = 0.99
        self.lr = 3e-4

        # data parameters
        self.drop_last = True
        self.batch_size = 128 # batch size is 128

        self.Context_Cont = Configs.Context_Cont_configs()
        self.TC = Configs.TC()
        self.augmentation = Configs.augmentations()



class Args:
      experiment_description = 'Exp1'  #default='Exp1', type=str, help='Experiment Description'
      run_description = 'run1'  #default='run1', type=str, help='Experiment Description'
      seed = 0  #default=0, type=int, help='seed value'
      training_mode = 'self_supervised'   #default='self_supervised', type=str, help='Modes of choice: random_init, supervised, self_supervised, fine_tune, train_linear'
      selected_dataset = 'ElectricMotorFaults'    #default='ECG5000', type=str, help='Dataset of choice: ECG200, ECG5000, CinCECGTorso, CentrifugalPump, HydraulicPump, ElectricMotorFaults'
      logs_save_dir = '/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC/logs'
      home_path = home_dir #default=home_dir, type=str, help='Project home directory'

args = Args()
configs = Configs() #see config_files folder for recommended configs

#device = torch.device('cuda') #default='cuda', type=str, help='cpu or cuda'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
experiment_description = args.experiment_description
data_type = args.selected_dataset
method = 'TS-TCC'
training_mode = args.training_mode
run_description = args.run_description

logs_save_dir = args.logs_save_dir
os.makedirs(logs_save_dir, exist_ok=True)


# ##### fix random seeds for reproducibility ########
SEED = args.seed
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
#####################################################

experiment_log_dir = os.path.join(logs_save_dir, experiment_description, run_description, training_mode + f"_seed_{SEED}")
os.makedirs(experiment_log_dir, exist_ok=True)

# loop through domains
counter = 0
src_counter = 0


# Logging
log_file_name = os.path.join(experiment_log_dir, f"logs_{datetime.now().strftime('%d_%m_%Y_%H_%M_%S')}.log")
logger = _logger(log_file_name)
logger.debug("=" * 45)
logger.debug(f'Dataset: {data_type}')
logger.debug(f'Method:  {method}')
logger.debug(f'Mode:    {training_mode}')
logger.debug("=" * 45)

# Load datasets
data_path = f"./data/{data_type}"
train_dl, valid_dl, test_dl = data_generator(data_path, configs, training_mode)
logger.debug("Data loaded ...")

# Load Model
model = base_Model(configs).to(device)
temporal_contr_model = TC(configs, device).to(device)

if training_mode == "fine_tune":
    # load saved model of this experiment
    load_from = os.path.join(os.path.join(logs_save_dir, experiment_description, run_description, f"self_supervised_seed_{SEED}", "saved_models"))
    chkpoint = torch.load(os.path.join(load_from, "ckp_last.pt"), map_location=device)
    pretrained_dict = chkpoint["model_state_dict"]
    model_dict = model.state_dict()
    del_list = ['logits']
    pretrained_dict_copy = pretrained_dict.copy()
    for i in pretrained_dict_copy.keys():
        for j in del_list:
            if j in i:
                del pretrained_dict[i]
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

if training_mode == "train_linear" or "tl" in training_mode:
    load_from = os.path.join(os.path.join(logs_save_dir, experiment_description, run_description, f"self_supervised_seed_{SEED}", "saved_models"))
    chkpoint = torch.load(os.path.join(load_from, "ckp_last.pt"), map_location=device)
    pretrained_dict = chkpoint["model_state_dict"]
    model_dict = model.state_dict()

    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

    # delete these parameters (Ex: the linear layer at the end)
    del_list = ['logits']
    pretrained_dict_copy = pretrained_dict.copy()
    for i in pretrained_dict_copy.keys():
        for j in del_list:
            if j in i:
                del pretrained_dict[i]

    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    set_requires_grad(model, pretrained_dict, requires_grad=False)  # Freeze everything except last layer.

if training_mode == "random_init":
    model_dict = model.state_dict()

    # delete all the parameters except for logits
    del_list = ['logits']
    pretrained_dict_copy = model_dict.copy()
    for i in pretrained_dict_copy.keys():
        for j in del_list:
            if j in i:
                del model_dict[i]
    set_requires_grad(model, model_dict, requires_grad=False)  # Freeze everything except last layer.



model_optimizer = torch.optim.Adam(model.parameters(), lr=configs.lr, betas=(configs.beta1, configs.beta2), weight_decay=3e-4)
temporal_contr_optimizer = torch.optim.Adam(temporal_contr_model.parameters(), lr=configs.lr, betas=(configs.beta1, configs.beta2), weight_decay=3e-4)


if training_mode == "self_supervised":  # to do it only once
    copy_Files(os.path.join(logs_save_dir, experiment_description, run_description), data_type)

# Trainer
Trainer(model, temporal_contr_model, model_optimizer, temporal_contr_optimizer, train_dl, valid_dl, test_dl, device, logger, configs, experiment_log_dir, training_mode)

if training_mode != "self_supervised":
    # Testing
    #outs = model_evaluate(model, temporal_contr_model, test_dl, device, training_mode)
    #total_loss, total_acc, pred_labels, true_labels = outs
    #total_loss, total_acc, f1, recall, precision, pred_labels, true_labels = model_evaluate(model, temporal_contr_model, test_dl, device, training_mode)
    total_loss, total_acc, f1, recall, precision, pred_labels, true_labels = model_evaluate(model, temporal_contr_model, test_dl, device, training_mode)
    _calc_metrics(pred_labels, true_labels, experiment_log_dir, args.home_path)

print(f"Training time is : {datetime.now()-start_time}")

DEBUG:/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC/logs/Exp1/run1/self_supervised_seed_0/logs_24_06_2024_02_54_05.log:=============================================


Dataset: ElectricMotorFaults


DEBUG:/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC/logs/Exp1/run1/self_supervised_seed_0/logs_24_06_2024_02_54_05.log:Dataset: ElectricMotorFaults


Method:  TS-TCC


DEBUG:/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC/logs/Exp1/run1/self_supervised_seed_0/logs_24_06_2024_02_54_05.log:Method:  TS-TCC


...


Epoch : 40
Train Loss     : 13.5876	 | 	Train Accuracy     : 0.0000
Valid Loss     : 0.0000	 | 	Valid Accuracy     : 0.0000


DEBUG:/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC/logs/Exp1/run1/self_supervised_seed_0/logs_24_06_2024_02_54_05.log:
Epoch : 40
Train Loss     : 13.5876	 | 	Train Accuracy     : 0.0000
Valid Loss     : 0.0000	 | 	Valid Accuracy     : 0.0000



################## Training is Done! #########################


DEBUG:/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/TSTCC/logs/Exp1/run1/self_supervised_seed_0/logs_24_06_2024_02_54_05.log:
################## Training is Done! #########################


Training time is : 0:00:58.911426


In [6]:
from google.colab import runtime
runtime.unassign()